In [182]:
import pandas as pd
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
import cookielib
import urllib2
import requests
import re
import json

In [77]:
def make_soup(url):
    #url = 'https://www.washingtonpost.com/politics'
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html.decode('utf-8', 'ignore'), 'html.parser')
    results = soup.find_all('h3', attrs = {'class': ''})
    print response.status_code
    return results

In [216]:
def make_dict(results):
    # initialize lists
    titles = []
    dates = []
    links = []
    bodies = []
    authors = []
    sections = []

    for x in results:
        
        link = x.find('a')['href']
        match1 = re.search('^https://www.washingtonpost.com/video', link)
        match2 = re.search('story.html', link)
        if not match1 and match2:
            links.append(link)

    # resoup link
    for link in links:
        response = requests.get(link)
        html = response.content
        new_soup = BeautifulSoup(html.decode('utf-8', 'ignore'), 'html.parser')

        # get title
        title = new_soup.find('title').text.strip().replace(' - The Washington Post','')
        titles.append(title)

        # get body
        all_p = ''
        body = new_soup.find('article', itemprop = 'articleBody')
        paragraphs = body.find_all('p')
        for p in paragraphs:
            new_p = p.text.strip().encode('ascii', 'ignore')
            all_p = all_p + new_p   
        bodies.append(all_p)

        # get date
        date = new_soup.find('span', attrs = {'itemprop': 'datePublished'})['content']
        dates.append(date)

        # get section
        
        scripts = new_soup.find_all('script')
        pattern = 'var commercialNode'

        for script in scripts:
            #if (pattern.match(str(script.string))):
            match = re.search(pattern, str(script))
            if match:
                text = script.text.strip()
                text_list = text.split(';')
                for x in text_list:
                    match = re.search(pattern, x)
                    if match:
                        section = x.replace('var commercialNode=', '').replace('"','')
                        print section
                        
     
        sections.append(section)
        
        # get author
        author = ''
        author_list = new_soup.find_all('span', attrs = {'itemprop': 'name'})
        for a in author_list:
            a = a.text.strip()
            author = author + ',' + a
        authors.append(author)
        
        data_dict = {
            'title': titles, 'link': links, 'author': authors,
            'body': bodies, 'section': sections, 'date': dates
        }
    return data_dict

In [217]:
def make_df(data_dict):
    df = pd.DataFrame(data_dict)
    return df

In [218]:
results = make_soup('http://www.washingtonpost.com/opinions')
my_dict = make_dict(results)
my_df = make_df(my_dict)

200
opinions
opinions
opinions
opinions
opinions
opinions
opinions
opinions
opinions
opinions
opinions
opinions
opinions
opinions
opinions
opinions
opinions
opinions
opinions


In [219]:
my_df.head()

,author,body,date,link,section,title
0,",Ruth Marcus,Ruth Marcus",This is a column on a subject of broad public ...,2017-02-03T07:57-500,https://www.washingtonpost.com/opinions/if-you...,opinions,"If you’re reading this, Justice Kennedy, pleas..."
1,",George F. Will,George F. Will",Tight labor markets shrink income inequality b...,2017-02-03T07:57-500,https://www.washingtonpost.com/opinions/want-t...,opinions,Want to reduce inequality? Try the Black Death.
2,",Kathleen Parker,Kathleen Parker",To review the lefts reaction to Supreme Court ...,2017-02-03T07:56-500,https://www.washingtonpost.com/opinions/the-le...,opinions,The left’s boogeyman vision of Gorsuch
3,",Rachel Manteuffel,Rachel Manteuffel",Rachel Manteuffel works in The Posts Editorial...,2017-02-03T07:55-500,https://www.washingtonpost.com/opinions/dear-r...,opinions,Dear Ruth Bader Ginsburg: If you need anything...
4,",Henry Olsen,Henry Olsen",Henry Olsen is a senior fellow at the Ethics a...,2017-02-03T07:55-500,https://www.washingtonpost.com/opinions/trumps...,opinions,"Trump’s election is the last, best hope to re-..."


In [147]:
url = links[1]

In [172]:
response = requests.get(url)
html = response.content
new_soup = BeautifulSoup(html.decode('ascii', 'ignore'), 'html.parser')

In [213]:
scripts = new_soup.find_all('script')
pattern = 'var commercialNode'

for script in scripts:
    #if (pattern.match(str(script.string))):
    match = re.search(pattern, str(script))
    if match:
        text = script.text.strip()
        text_list = text.split(';')
        for x in text_list:
            match = re.search(pattern, x)
            if match:
                section = x.replace('var commercialNode=', '').replace('"','')
                print section

powerpost


In [162]:
scripts = new_soup.find_all('script')[2]

In [166]:
type(scripts)

bs4.element.Tag